<a href="https://colab.research.google.com/github/mike-a-yen/Kaggle/blob/master/booking_recsys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Booking.com RecSys Challenge 2021

- Given a sequence of stops on a trip, predict the next stop.

https://www.bookingchallenge.com/

# Pip install

In [ ]:
!pip install torch pytorch_lightning sklearn wget tqdm plotly torchmetrics wandb

# Imports

In [ ]:
!nvidia-smi

Mon Aug  9 22:33:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    33W / 250W |   1017MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from pathlib import Path
from collections import Counter, namedtuple
from typing import Optional
import zipfile

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.utils as utils
import torchmetrics
from tqdm import tqdm
import wandb
import wget

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger

%matplotlib notebook
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
DATA_DIR = Path('./data').resolve()
DATA_DIR.mkdir(exist_ok=True, parents=True)

## Utils

In [ ]:
class CityIndex:
    """
    city_index = CityIndex.from_list(train_df.city_id.tolist(), max_size=10_000)
    """
    def __init__(self, max_size: int = None) -> None:
        self.max_size = max_size
        self.counter = Counter()
        self.specials = []
        self.index = None
        self.reverse_index = []
        self._add_special('<pad>')
        self._add_special('<unk>')
        self._add_special('<bos>')
        self._add_special('<eos>')

    def __call__(self, ids, reverse=False):
        if reverse:
            return [self.reverse_index[i] for i in ids]
        return [self.index.get(i, self.unk_id) for i in ids]

    @classmethod
    def from_list(cls, city_ids, **kwargs):
        index = CityIndex(**kwargs)
        index.update(city_ids)
        index.build_index()
        return index

    def _add_special(self, city_id):
        self.counter[city_id] = float('inf')
        self.specials.append(city_id)

    def update(self, city_id):
        """Update counter with one or many cities."""
        if isinstance(city_id, int):
            self.counter.update([city_id])
        else:
            self.counter.update(city_id)

    def __getitem__(self, city_id):
        return self.index.get(city_id, self.unk_id)

    def build_index(self):
        self.index = dict()
        self.reverse_index = []
        size = self.max_size if self.max_size is not None else len(self.counter)
        for i, (city_id, count) in enumerate(self.counter.most_common(size)):
            self.index[city_id] = i
            while i >= len(self.reverse_index):
                self.reverse_index.append(None)
            self.reverse_index[i] = city_id

    @property
    def size(self):
        return len(self.index)

    @property
    def pad_id(self):
        return self.index['<pad>']

    @property
    def unk_id(self):
        return self.index['<unk>']
    
    @property
    def bos_id(self):
        return self.index['<bos>']
    
    @property
    def eos_id(self):
        return self.index['<eos>']

    @property
    def special_ids(self):
        return [self.index(i) for i in self.specials]

In [ ]:
def concatenate_groups(x):
    return x.tolist()


def collect_trip_df(df, city_index):
    df.sort_values('checkin', ascending=True, inplace=True)
    trip_df = df.groupby('utrip_id').agg(
        cities=('city_id', concatenate_groups),
        countries=('hotel_country', concatenate_groups),
        booker_countries=('booker_country', concatenate_groups),
        trip_start=('checkin', 'min'),
        trip_end=('checkout', 'max'),
        checkins=('checkin', concatenate_groups),
        checkouts=('checkout', concatenate_groups)
    )

    trip_df['destinations'] = trip_df.cities.apply(lambda x: [city_index.bos_id] + city_index(x))
    trip_df['country_destinations'] = trip_df.countries.apply(lambda x: [city_index.bos_id] + city_index(x))
    trip_df['checkin_months'] = trip_df.checkins.apply(lambda x: [city_index.bos_id] + [i.month for i in x])
    trip_df['days_from_trip_start'] = trip_df.apply(
        lambda row: [-1] + [np.log( (i - row.trip_start).days + 1 ) for i in row.checkins], axis=1)
    trip_df['days_till_trip_end'] = trip_df.apply(
        lambda row: [-1] + [np.log( (i - row.trip_end).days + 1 ) for i in row.checkins], axis=1)
    trip_df['days_from_last_stop'] = trip_df.checkins.apply(
        lambda x: [-1, 0] + [np.log( (x[i] - x[i-1]).days + 1 ) for i in range(1, len(x))])
    trip_df['checkin_dayofweek'] = trip_df.checkins.apply(
        lambda x: [7] + [i.dayofweek for i in x])
    trip_df['num_stops'] = trip_df.cities.apply(
        lambda x: [-1] + [np.log(i + 1) for i, _ in enumerate(x)])
    trip_df['stop_length_days'] = trip_df.apply(
        lambda row: [0] + [np.log( (co - ci).days + 1 ) for ci, co in zip(row.checkins, row.checkouts)], axis=1)
    trip_df['international_stop'] = trip_df.apply(
        lambda row: [0] + [int(row.booker_countries[0] != row.countries[0])] \
            + [int(row.countries[i] != row.countries[i - 1]) for i in range(1, len(row.countries))],
        axis=1)
    return trip_df

## Lightning Data

In [ ]:
sequential_features = [
            'destinations',
            'country_destinations',
            'checkin_months',
            'days_from_trip_start',
            'days_till_trip_end',
            'days_from_last_stop',
            'checkin_dayofweek',
            'num_stops',
            'stop_length_days',
            'international_stop'
        ]
pred_sample = namedtuple('pred_sample', sequential_features + ['maxlen'])
float_features = [
                'days_from_trip_start',
                'days_till_trip_end',
                'days_from_last_stop',
                'num_stops',
                'stop_length_days',
                'international_stop'
]


class TripLMDataset(utils.data.Dataset):
    def __init__(self, df, mode: str) -> None:
        self.df = df
        self.mode = mode

    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        X, Y, maxlen = [], None, 0
        for feature in sequential_features:
            data = torch.tensor(sample[feature])
            # when training trim and shift the inputs to create labels
            # when testing do nothing
            x = data if self.mode == 'test' else data[:-1]
            if feature == 'destinations':
                Y = torch.tensor( sample[feature][1:] + [0]) if self.mode == 'test' else data[1:]
                maxlen = len(x)
            X.append(x)
        X.append(maxlen)
        return pred_sample(*X) , Y

    def __len__(self):
        return self.df.shape[0]


pred_batch = namedtuple('pred_batch', 
                        [
                         'input_ids',
                         'country_ids',
                         'checkin_months',
                         'days_from_trip_start',
                         'days_till_trip_end',
                         'days_from_last_stop',
                         'checkin_dayofweek',
                         'num_stops',
                         'stop_length_days',
                         'international_stop',
                         'attention_mask'
                         ]
)


class BatchManager:
    def __init__(self, pad_id: int) -> None:
        self.pad_id = pad_id
        self.fields = list(pred_sample._fields)
        self.fields.pop(self.fields.index('maxlen'))
        self.field_dtypes = [torch.int64 if f not in float_features else torch.float32 for f in self.fields]

    def __call__(self, batch):
        N = len(batch)
        maxlen = max([sample[0].maxlen for sample in batch])
        input_ids = torch.zeros((N, maxlen), dtype=torch.int64) + self.pad_id
        X = [torch.zeros((N, maxlen), dtype=dtype) for field, dtype in zip(self.fields, self.field_dtypes)]
        mask = torch.zeros((N, maxlen), dtype=torch.int64)
        Y = torch.zeros((N, maxlen), dtype=torch.int64) + self.pad_id
        for i, (sample, y) in enumerate(batch):
            size = sample.maxlen
            for j, feature in enumerate(self.fields):
                x = getattr(sample, feature)
                X[j][i, -size:] = x
            mask[i, -size:] = 1
            Y[i, -size:] = y
        X.append(mask)
        return pred_batch(*X), Y


class DataModule(pl.LightningDataModule):
    def __init__(self, data_dir: Path, batch_size: int = 32, subsample: int = 0):
        super().__init__()
        self.wget_url = "https://035105f7-ae32-47b6-a25b-87af7924c7ea.filesusr.com/archives/3091be_233bace50f3f48fba40547a89443a96e.zip?dn=booking_dataset.zip"
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.subsample = subsample

    def prepare_data(self):
        zip_cache = self.data_dir / 'booking_dataset.zip'
        if not zip_cache.exists():
            print('Downloading data...')
            wget.download(self.wget_url, str(zip_cache))
            with zipfile.ZipFile(zip_cache, 'r') as zip_ref:
                zip_ref.extractall(self.data_dir)

    def setup(self, stage: Optional[str] = None):
        train_df = pd.read_csv(self.data_dir / 'train_set.csv').sort_values(by=['utrip_id','checkin'])
        train_df['checkin'] = pd.to_datetime(train_df.checkin)
        train_df['checkout'] = pd.to_datetime(train_df.checkout)
        if self.subsample > 0:
            train_df = train_df.iloc[0: self.subsample].copy()
        test_df = pd.read_csv(self.data_dir / 'test_set.csv').sort_values(by=['utrip_id','checkin'])
        test_df['checkin'] = pd.to_datetime(test_df.checkin)
        test_df['checkout'] = pd.to_datetime(test_df.checkout)

        print('Building index...')
        self.city_index = CityIndex.from_list(
            train_df.city_id.tolist() + train_df.hotel_country.tolist(),
            max_size=25_000
        )

        print('Preparing trips...')
        train_trip_df = collect_trip_df(train_df, self.city_index)
        train_trip_df, val_trip_df = train_test_split(train_trip_df, test_size=0.1)
        test_trip_df = collect_trip_df(test_df, self.city_index)

        self.train_ds = TripLMDataset(train_trip_df, mode='train')
        self.val_ds = TripLMDataset(val_trip_df, mode='val')
        self.test_ds = TripLMDataset(test_trip_df, mode='test')
        self.batch_manager = BatchManager(self.city_index.pad_id)
        print('Done.')

    def train_dataloader(self):
        return self._make_dataloader(self.train_ds, shuffle=True)

    def val_dataloader(self):
        return self._make_dataloader(self.val_ds, False)

    def test_dataloader(self):
        return self._make_dataloader(self.test_ds, False)

    def predict_dataloader(self):
        return self._make_dataloader(self.test_ds, False)

    def _make_dataloader(self, ds, shuffle = False):
        return utils.data.DataLoader(ds, collate_fn=self.batch_manager, batch_size=self.batch_size, shuffle=shuffle, pin_memory=True, num_workers=4)

In [ ]:
data_module = DataModule(DATA_DIR, batch_size=128, subsample=0)
data_module.prepare_data()
data_module.setup()

Building index...
Preparing trips...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning:

invalid value encountered in log

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning:

divide by zero encountered in log



Done.


## Lightning Models

In [ ]:
class TripModel(nn.Module):
    def __init__(self, num_cities, hidden_size, num_layers, numeric_features, dropout: float = 0.1, max_trip_size: int = 128, mode: str = 'GRU'):
        super().__init__()
        self.num_cities = num_cities
        self.hidden_size = hidden_size
        self.numeric_features = numeric_features

        self.embedding_layer = nn.Embedding(num_cities, hidden_size)
        self.months_layer = nn.Embedding(13, hidden_size)
        self.dayofweek_layer = nn.Embedding(8, hidden_size)
        if len(self.numeric_features) > 0:
            self.numeric_lin = nn.Linear(len(self.numeric_features), hidden_size)
        if mode == 'GRU':
            self.encoder = nn.ModuleList([
                nn.GRU((2 + (len(self.numeric_features) > 0)) * hidden_size, hidden_size, 1, dropout=0.1, batch_first=True),
                nn.GRU(hidden_size, hidden_size, num_layers, dropout=0.1, batch_first=True)
            ])
        elif mode == 'LSTM':
            self.encoder = nn.ModuleList([
                nn.LSTM((2 + (len(self.numeric_features) > 0)) * hidden_size, hidden_size, 1, dropout=0.1, batch_first=True),
                nn.LSTM(hidden_size, hidden_size, num_layers, dropout=0.1, batch_first=True)
            ])
        self.drop = nn.ModuleList([nn.Dropout(dropout) for _ in self.encoder])
        self.lin = nn.Linear(hidden_size, num_cities, bias=False)

    def forward(self, X):
        months = self.months_layer(X.checkin_months)
        days = self.dayofweek_layer(X.checkin_dayofweek)
        time_emb = months + days
        loc_emb = self.embedding_layer(X.input_ids) + self.embedding_layer(X.country_ids)
        emb = torch.cat((loc_emb, time_emb), dim=-1)
        if len(self.numeric_features) > 0:
            trip_details = torch.stack([getattr(X, f) for f in self.numeric_features], dim=-1)
            detail_emb = self.numeric_lin(trip_details)
            emb = torch.cat((emb, detail_emb), dim=-1)
        for (layer, drop) in zip(self.encoder, self.drop):
            emb, *_ = layer(emb)
            emb = drop(emb)
        return self.lin(emb)


class LightningTripModel(pl.LightningModule):
    def __init__(self, *args, **kwargs):
        super().__init__()
        self.model = TripModel(*args, **kwargs)
        self.metrics = nn.ModuleDict({
            'top4_accuracy': torchmetrics.Accuracy(top_k=4, ignore_index=0)
        })
        self.eval_metric = torchmetrics.Accuracy(top_k=4, ignore_index=0)
        self.lr = 0.01
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=0)

    def forward(self, X):
        return self.model.forward(X)

    def forward_with_loss(self, xb, yb):
        logits = self.forward(xb)
        cost = self.loss_fn(logits.view(-1, self.model.num_cities), yb.view(-1))
        return logits, cost

    def training_step(self, batch, batch_idx):
        xb, yb = batch
        y_hat, cost = self.forward_with_loss(xb, yb)
        return cost

    def validation_step(self, batch, batch_idx):
        xb, yb = batch
        logits, cost = self.forward_with_loss(xb, yb)
        self.log("val_loss", cost)
        probs = logits.softmax(dim=-1)
        for name, metric in self.metrics.items():
            metric(probs.view(-1, self.model.num_cities), yb.view(-1))
        self.eval_metric(probs[:, -1], yb[:, -1])  # accuracy on last destination in trip
        return cost

    def validation_epoch_end(self, outs):
        for name, metric in self.metrics.items():
            self.log(f'{name}', metric.compute())
        self.log(f'eval_top4_accuracy', self.eval_metric.compute())

    def predict_step(self, batch, batch_idx):
        xb, yb = batch
        logits = self.forward(xb)
        return logits

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

# Train

In [ ]:
config = {
    'model': {
        'num_cities': data_module.city_index.size,
        'hidden_size': 128,
        'num_layers': 2,
        'numeric_features': ['days_from_trip_start', 'days_from_last_stop', 'num_stops', 'stop_length_days', 'international_stop'],
        'dropout': 0.1,
        'mode': 'GRU'
    },
    'trainer': {
        'max_epochs': 8,
        'auto_lr_find': True,
        'gpus': torch.cuda.device_count(),
        'fast_dev_run': False
    }
}

In [ ]:
pl_model = LightningTripModel(**config['model'])

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1



In [ ]:
use_wandb = True
if use_wandb:
    run = wandb.init(project='booking-recsys', tags=[config['model']['mode']], config=config)
    logger = WandbLogger(experiment=run)
else:
    logger = TensorBoardLogger(DATA_DIR / 'tb_logs', name=mode)

callbacks = [
             EarlyStopping(monitor="val_loss")
]
trainer = pl.Trainer(logger=logger, callbacks=callbacks, **config['trainer'])

wandb: Currently logged in as: mike-a-yen (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.11.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
if config['trainer']['auto_lr_find']:
    results = trainer.tune(pl_model, datamodule=data_module)['lr_find']
    fig = results.plot(suggest=True)
    fig.show()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning:

DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type             | Params
-------------------------------------------------
0 | model       | TripModel        | 6.8 M 
1 | metrics     | ModuleDict       | 0     
2 | eval_metric | Accuracy         | 0     
3 | loss_fn     | CrossEntropyLoss | 0     
-------------------------------------------------
6.8 M     Trainable params
0         Non-trainable params
6.8 M     Total params
27.196    Total estimated model params size (MB)


Restoring states from the checkpoint file at /content/lr_find_temp_model.ckpt
Restored all states from the checkpoint file at /content/lr_find_temp_model.ckpt
Learning rate set to 0.003311311214825908


<IPython.core.display.Javascript object>

In [ ]:
trainer.fit(pl_model, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning:

DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type             | Params
-------------------------------------------------
0 | model       | TripModel        | 6.8 M 
1 | metrics     | ModuleDict       | 0     
2 | eval_metric | Accuracy         | 0     
3 | loss_fn     | CrossEntropyLoss | 0     
-------------------------------------------------
6.8 M     Trainable params
0         Non-trainable params
6.8 M     Total params
27.196    Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning:

DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.



# Evaluate

In [ ]:
gt_df = pd.read_csv(DATA_DIR / 'ground_truth.csv')
gt_df.head()

,utrip_id,city_id,hotel_country
0,1038944_1,54085,Sokovia
1,1068715_1,29319,Cobra Island
2,1075528_1,55763,Bozatta
3,1110462_4,11930,Alvonia
4,1132565_1,58659,Axphain


In [ ]:
test_df = pd.read_csv(DATA_DIR / 'test_set.csv')
test_df.head(10)

,user_id,checkin,checkout,device_class,affiliate_id,booker_country,utrip_id,city_id,hotel_country
0,1000066,2016-07-21,2016-07-23,desktop,9924,Gondal,1000066_2,56430,Urkesh
1,1000066,2016-07-23,2016-07-25,desktop,9924,Gondal,1000066_2,41971,Urkesh
2,1000066,2016-07-25,2016-07-28,desktop,9924,Gondal,1000066_2,5797,Urkesh
3,1000066,2016-07-28,2016-07-31,mobile,2436,Gondal,1000066_2,0,NaN
4,1000270,2016-02-08,2016-02-09,mobile,9452,The Devilfire Empire,1000270_1,50075,The Devilfire Empire
5,1000270,2016-02-09,2016-02-10,desktop,116,The Devilfire Empire,1000270_1,29207,Cobra Island
6,1000270,2016-02-10,2016-02-19,desktop,9924,The Devilfire Empire,1000270_1,44768,Cobra Island
7,1000270,2016-02-19,2016-02-20,desktop,9924,The Devilfire Empire,1000270_1,0,NaN
8,1000441,2016-05-29,2016-06-01,desktop,9924,Bartovia,1000441_1,47759,Osterlich
9,1000441,2016-06-01,2016-06-02,desktop,9924,Bartovia,1000441_1,46411,Osterlich


In [ ]:
def predict_on_ds(ds, pl_model, data_module, topk: int = 4):
    dl = utils.data.DataLoader(
        ds, collate_fn=data_module.batch_manager, batch_size=128, shuffle=False,
        num_workers=4
    )
    city_predictions = []
    special_token_ids = torch.tensor([data_module.city_index[i] for i in data_module.city_index.specials], device=pl_model.device)
    with torch.no_grad():
        for xb, yb in tqdm(dl, desc='Predicting'):
            xb_device = pred_batch(*[getattr(xb, f).to(pl_model.device) for f in xb._fields])
            logits = pl_model(xb_device)[:, -2]
            logit_mask = torch.zeros_like(logits)
            logit_mask[:, special_token_ids] -= 1e9
            logits += logit_mask
            probs = logits.softmax(dim=-1)
            preds = torch.topk(probs, topk, dim=-1)
            prediction_index = preds.indices.tolist()
            city_predictions += [data_module.city_index(row, reverse=True) for row in prediction_index]
    return {'predictions': city_predictions, 'utrip_id': ds.df.index.tolist()}

In [ ]:
results = predict_on_ds(data_module.test_ds, pl_model, data_module)

# make the results df
payload = {'utrip_id': results['utrip_id']}
for i in range(4):
    payload[f'city_id_{i+1}'] = [row[i] for row in results['predictions']]
df = pd.DataFrame(payload)
df = df.merge(gt_df, how='left', left_on='utrip_id', right_on='utrip_id')
df['is_correct'] = df.apply(lambda row: row.city_id in [row[f'city_id_{i+1}'] for i in range(4)], axis=1).astype(int)

# compute accuracy
acc = df.is_correct.mean()
print(f'Top 4 Accuracy: {100 * acc:0.2f}%')
df.head()

Predicting: 100%|██████████| 553/553 [02:17<00:00,  4.01it/s]


Top 4 Accuracy: 47.08%


,utrip_id,city_id_1,city_id_2,city_id_3,city_id_4,city_id,hotel_country,is_correct
0,1000066_2,5797,39200,30018,46854,11543,Urkesh,0
1,1000270_1,27404,32821,38677,21328,29207,Cobra Island,0
2,1000441_1,35160,13260,8935,56421,35160,Osterlich,1
3,100048_1,26235,13530,45030,13150,33394,Alvonia,0
4,1000543_1,44869,52818,25390,67353,52818,Elbonia,1


In [ ]:
trainer.logger.experiment.log({'final_top4_accuracy': acc})

In [ ]:
trainer.logger.experiment.finish()

val_loss,4.78485
top4_accuracy,0.35258
eval_top4_accuracy,0.46999
epoch,7
trainer/global_step,12248
_runtime,645
_timestamp,1628549251
_step,8
final_top4_accuracy,0.47083


val_loss,█▄▂▁▁▁▁▁
top4_accuracy,▁▄▅▆▇▇██
eval_top4_accuracy,▁▄▅▆▇▇██
epoch,▁▂▃▄▅▆▇█
trainer/global_step,▁▂▃▄▅▆▇██
_runtime,▁▂▃▃▄▅▆▆█
_timestamp,▁▂▃▃▄▅▆▆█
_step,▁▂▃▄▅▅▆▇█
final_top4_accuracy,▁
